In [2]:
from transformers import ViltProcessor, ViltForQuestionAnswering
import requests
from PIL import Image

# prepare image + question
path = "/content/0002.PNG"
image = Image.open(path).convert('RGB')
text = "Is this nurse putting on gloves?"

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

# prepare inputs
encoding = processor(image, text, return_tensors="pt")

# forward pass
outputs = model(**encoding)
logits = outputs.logits
idx = logits.argmax(-1).item()
print("Predicted answer:", model.config.id2label[idx])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/136k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/470M [00:00<?, ?B/s]

Predicted answer: yes


In [5]:
!pip install -q transformers

In [7]:
from transformers import ViltForQuestionAnswering
import torch

from transformers import ViltConfig

config = ViltConfig.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

ViltForQuestionAnswering(
  (vilt): ViltModel(
    (embeddings): ViltEmbeddings(
      (text_embeddings): TextEmbeddings(
        (word_embeddings): Embedding(30522, 768)
        (position_embeddings): Embedding(40, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (patch_embeddings): ViltPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
      )
      (token_type_embeddings): Embedding(2, 768)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViltEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViltLayer(
          (attention): ViltAttention(
            (attention): ViltSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=76

In [9]:
from google.colab import files

import zipfile
import io
import os

f = files.upload()

# Get the uploaded zip archive file
uploaded_zip_filename = next(iter(f))

# Extract the contents of the zip archive
with zipfile.ZipFile(io.BytesIO(f[uploaded_zip_filename]), 'r') as zip_ref:
    zip_ref.extractall('/content')

# Function to list all files in a directory recursively
def list_files_recursive(root_dir):
    file_names = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            # Get the filename without the directory path
            file_name = os.path.basename(file)
            file_names.append(file_name)
    return file_names

Saving test_images.zip to test_images.zip


In [10]:
# Get the list of all files in the '/content/' directory and its subdirectories
test_file_names = list_files_recursive('/content/test_images')

# Print the list of file names
print(test_file_names)

['0292.PNG', '40.png', '0253.PNG', '0287.PNG', '32.png', '27.png', '0235.PNG', '39.png', '70.png', '0251.PNG', '36.png', '0274.PNG', '0238.PNG', '26.png', '0286.PNG', '0250.PNG', '48.png', '31.png', '0232.PNG', '0260.PNG', '67.png', '1.png', '0276.PNG', '41.png', '0281.PNG', '0278.PNG', '0244.PNG', '294.PNG', '0267.PNG', '0229.PNG', '66.png', '0282.PNG', '25.png', '0288.PNG', '295.PNG', '47.png', '72.png', '23.png', '0236.PNG', '0266.PNG', '35.png', '13.png', '0272.PNG', '0283.PNG', '0234.PNG', '0269.PNG', '59.png', '14.png', '0231.PNG', '0275.PNG', '0264.PNG', '16.png', '71.png', '0268.PNG', '0256.PNG', '18.png', '0290.PNG', '38.png', '30.png', '37.png', '65.png', '0254.PNG', '68.png', '0227.PNG', '0271.PNG', '28.png', '0289.PNG', '0285.PNG', '24.png', '0270.PNG', '0228.PNG', '69.png', '58.png', '3.png', '0233.PNG', '17.png', '0230.PNG', '45.png', '33.png', '0246.PNG', '0273.PNG', '60.png', '64.png', '0261.PNG', '15.png', '34.png', '0291.PNG', '62.png', '63.png', '0249.PNG', '0239.PNG

In [11]:
print("Test image count: ", len(test_file_names))

Test image count:  101


In [12]:
import json

# Read annotations
f = open('/content/test_annotations.json')

# Return JSON object as dictionary
test_annotations = json.load(f)
print(test_annotations.keys())

annotations = test_annotations['annotations']

f = open('/content/test_questions.json')

# Return JSON object as dictionary
test_questions = json.load(f)
print(test_questions.keys())

questions = test_questions['questions']

dict_keys(['info', 'license', 'data_subtype', 'annotations'])
dict_keys(['info', 'license', 'data_subtype', 'tast_type', 'data_type', 'questions'])


In [13]:
print(len(questions))
print(len(annotations))

104
99


In [14]:
classes = {}
for item in questions:
  question = item['question']
  if question in classes:
    classes[question] += 1
  else:
    classes[question] = 1

for key in classes:
  print(key)
print(len(classes))

is this nurse putting on latex gloves?
is this nurse raising the head of the patient bed?
is this nurse preparing iv hang medication?
is this nurse having a phone call?
is this nurse putting the inhaler on the patient??
is this nurse administering iv hang medication?
is this nurse rubbing alcohol swab on iv pump?
is this nurse taking the patient's pulse?
is this nurse taking the patient's temperature?
is this nurse listening to the patient's chest?
is this nurse administering iv push medication?
11


In [15]:
def question_id_to_question(questions):
  dic = {}
  for question in questions:
    question_id = question["question_id"]
    question = question["question"]
    dic[question_id] = question
  return dic

dic = question_id_to_question(questions)
print(dic)

print(dic[171])

{11: 'is this nurse putting on latex gloves?', 31: 'is this nurse putting on latex gloves?', 131: 'is this nurse putting on latex gloves?', 141: 'is this nurse putting on latex gloves?', 151: 'is this nurse raising the head of the patient bed?', 161: 'is this nurse raising the head of the patient bed?', 171: 'is this nurse putting on latex gloves?', 181: 'is this nurse putting on latex gloves?', 231: 'is this nurse preparing iv hang medication?', 241: 'is this nurse preparing iv hang medication?', 251: 'is this nurse preparing iv hang medication?', 261: 'is this nurse preparing iv hang medication?', 271: 'is this nurse preparing iv hang medication?', 281: 'is this nurse preparing iv hang medication?', 291: 'is this nurse preparing iv hang medication?', 301: 'is this nurse preparing iv hang medication?', 311: 'is this nurse preparing iv hang medication?', 321: 'is this nurse preparing iv hang medication?', 331: 'is this nurse preparing iv hang medication?', 341: 'is this nurse preparing

In [16]:
from typing import Optional
import re

# Assuming you have a regex pattern for matching the filename
filename_re = re.compile(r'^(\d+)_.*$')

def id_from_filename(filename: str) -> Optional[int]:
    # Strip leading zeros from the filename
    filename_without_zeros = filename.lstrip('0')

    # Remove the trailing ".jpg" if present
    filename_without_extension = filename_without_zeros.replace('.PNG', '')
    filename_without_extension = filename_without_extension.replace('.png', '')

    return int(filename_without_extension)

# Example usage
filename = '0123.PNG'
id = id_from_filename(filename)
print(id)

123


In [30]:
correct_predictions = 0
total_predictions = 0
res = {}
for i in range(0,99):
    total_predictions += 1
    file_name = test_file_names[i]
    image_id = annotations[i]["image_id"]
    question_id = annotations[i]["question_id"]
    question = (dic[question_id])
    if question == "is this nurse rubbing alcohol swab on iv pump?" or question == "is this nurse taking the patient's temperature?":
      continue
    #print()
    # print(file_name)
    file_path = "/content/test_images/" + file_name
    # print(file_path)
    question_id = (id_from_filename(file_name) * 10) + 1
    # print(question_id)
    question = (dic[question_id])
    # print(annotations[i]["answers"][0]["answer"])
    image = Image.open(file_path).convert('RGB')

    # Display the image
    #display(Image.open(file_path).convert('RGB'))
    # prepare inputs
    # print("question: ", question)
    encoding = processor(image, question, return_tensors="pt")

    # forward pass
    outputs = model(**encoding)
    logits = outputs.logits
    idx = logits.argmax(-1).item()
    prediction = model.config.id2label[idx]

    #print("Predicted answer:", prediction)
    #print("Ground truth:", annotations[i]["answers"][0]["answer"])

    if question in res:
      if prediction == annotations[i]["answers"][0]["answer"]:
        correct_predictions += 1
        res[question].append(1)
      else:
        res[question].append(0)
    else:
      if prediction == annotations[i]["answers"][0]["answer"]:
        correct_predictions += 1
        res[question] = [1]
      else:
        res[question] = [0]

print(res)

{'is this nurse preparing iv hang medication?': [0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0], 'is this nurse listening to patients chest?': [1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1], 'is this nurse putting on latex gloves?': [0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1], "is this nurse taking the patient's pulse?": [1, 1, 1, 0, 0, 0, 1, 0], 'is this nurse having a phone call?': [0, 0, 1, 1, 1, 0], "is this nurse taking the patient's temperature?": [1, 0], 'is this nurse putting the inhaler on the patient??': [0, 1, 0, 1, 1, 1, 0, 0], 'is this nurse administering iv hang medication?': [1, 0]}


In [31]:
for key in res:
  print(key, ": ", res[key])

is this nurse preparing iv hang medication? :  [0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0]
is this nurse listening to patients chest? :  [1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1]
is this nurse putting on latex gloves? :  [0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1]
is this nurse taking the patient's pulse? :  [1, 1, 1, 0, 0, 0, 1, 0]
is this nurse having a phone call? :  [0, 0, 1, 1, 1, 0]
is this nurse taking the patient's temperature? :  [1, 0]
is this nurse putting the inhaler on the patient?? :  [0, 1, 0, 1, 1, 1, 0, 0]
is this nurse administering iv hang medication? :  [1, 0]


In [32]:
accuracy = (correct_predictions / total_predictions) * 100
print(accuracy)

54.54545454545454


In [33]:
map = 0
for key in res:
  values = res[key]
  # Calculate the sum of the values
  #print(values)
  sum_values = sum(values)

  # Calculate the average
  average = sum_values / len(values)

  #print("Average:", average)
  map += average
  print(key, ": ", average)

map = map/len(res)
print(map * 100)

is this nurse preparing iv hang medication? :  0.6363636363636364
is this nurse listening to patients chest? :  0.5
is this nurse putting on latex gloves? :  0.5789473684210527
is this nurse taking the patient's pulse? :  0.5
is this nurse having a phone call? :  0.5
is this nurse taking the patient's temperature? :  0.5
is this nurse putting the inhaler on the patient?? :  0.5
is this nurse administering iv hang medication? :  0.5
52.69138755980861


In [24]:
import json

# Read annotations
f = open('/content/annotations.json')

# Return JSON object as dictionary
annotations = json.load(f)
print(annotations.keys())

annotations = annotations['annotations']

f = open('/content/questions.json')

# Return JSON object as dictionary
questions = json.load(f)
print(questions.keys())

questions = questions['questions']

dict_keys(['info', 'license', 'data_subtype', 'annotations'])
dict_keys(['info', 'license', 'data_subtype', 'tast_type', 'data_type', 'questions'])


In [25]:
def question_id_to_question(questions):
  dic = {}
  for question in questions:
    question_id = question["question_id"]
    question = question["question"]
    dic[question_id] = question
  return dic

dic = question_id_to_question(questions)
print(dic)

print(dic[171])

{11: 'is this nurse putting on latex gloves?', 21: 'is this nurse putting on latex gloves?', 81: 'is this nurse putting on latex gloves?', 71: 'is this nurse putting on latex gloves?', 61: 'is this nurse putting on latex gloves?', 51: 'is this nurse putting on latex gloves?', 41: 'is this nurse putting on latex gloves?', 31: 'is this nurse putting on latex gloves?', 91: 'is this nurse listening to patients chest?', 101: 'is this nurse listening to patients chest?', 111: 'is this nurse listening to patients chest?', 121: 'is this nurse listening to patients chest?', 151: 'is this nurse listening to patients chest?', 161: 'is this nurse listening to patients chest?', 131: 'is this nurse putting on latex gloves?', 141: 'is this nurse putting on latex gloves?', 171: 'is this nurse putting on latex gloves?', 181: 'is this nurse putting on latex gloves?', 191: 'is this nurse listening to patients chest?', 201: 'is this nurse listening to patients chest?', 231: 'is this nurse listening to pat

In [26]:
classes = {}
for item in questions:
  question = item['question']
  if question in classes:
    classes[question] += 1
  else:
    classes[question] = 1

for key in classes:
  print(key)
print(len(classes))

is this nurse putting on latex gloves?
is this nurse listening to patients chest?
is this nurse raising the head of the patient bed?
is this nurse having a phone call?
is this nurse administering oral medication?
is this nurse administering iv push medication?
is this nurse rubbing alcohol swab on patient's skin?
is this nurse poking patient's arm?
is this nurse administering iv hang medication?
is this nurse preparing iv hang medication?
is this nurse rubbing alcohol swab on iv pump?
is this nurse putting the inhaler on the patient??
is this nurse taking the patient's pulse?
is this nurse taking the patient's temperature?
is this nurse listening to the patient's chest?
15


In [27]:
yes_no_dic = {}
for item in annotations:
  answer = item["multiple_choice_answer"]
  question_id = item["question_id"]
  question = dic[question_id]
  if question in yes_no_dic:
    if answer == "yes":
      yes_no_dic[question]["yes"] += 1
    if answer == "no":
      yes_no_dic[question]["no"] += 1
  else:
    if answer == "yes":
      yes_no_dic[question] = {"yes": 1, "no": 0}
    if answer == "no":
      yes_no_dic[question]  = {"yes": 0, "no": 1}

print(yes_no_dic)

{'is this nurse putting on latex gloves?': {'yes': 36, 'no': 24}, 'is this nurse listening to patients chest?': {'yes': 23, 'no': 54}, 'is this nurse raising the head of the patient bed?': {'yes': 6, 'no': 4}, 'is this nurse having a phone call?': {'yes': 13, 'no': 12}, 'is this nurse administering oral medication?': {'yes': 1, 'no': 3}, 'is this nurse administering iv push medication?': {'yes': 32, 'no': 12}, "is this nurse rubbing alcohol swab on patient's skin?": {'yes': 4, 'no': 2}, "is this nurse poking patient's arm?": {'yes': 4, 'no': 2}, 'is this nurse administering iv hang medication?': {'yes': 8, 'no': 4}, 'is this nurse preparing iv hang medication?': {'yes': 21, 'no': 12}, 'is this nurse rubbing alcohol swab on iv pump?': {'yes': 3, 'no': 3}, 'is this nurse putting the inhaler on the patient??': {'yes': 9, 'no': 4}, "is this nurse taking the patient's pulse?": {'yes': 4, 'no': 4}, "is this nurse taking the patient's temperature?": {'yes': 1, 'no': 1}, "is this nurse listeni

In [28]:
for key in yes_no_dic:
  print(key, ": ", yes_no_dic[key])

is this nurse putting on latex gloves? :  {'yes': 36, 'no': 24}
is this nurse listening to patients chest? :  {'yes': 23, 'no': 54}
is this nurse raising the head of the patient bed? :  {'yes': 6, 'no': 4}
is this nurse having a phone call? :  {'yes': 13, 'no': 12}
is this nurse administering oral medication? :  {'yes': 1, 'no': 3}
is this nurse administering iv push medication? :  {'yes': 32, 'no': 12}
is this nurse rubbing alcohol swab on patient's skin? :  {'yes': 4, 'no': 2}
is this nurse poking patient's arm? :  {'yes': 4, 'no': 2}
is this nurse administering iv hang medication? :  {'yes': 8, 'no': 4}
is this nurse preparing iv hang medication? :  {'yes': 21, 'no': 12}
is this nurse rubbing alcohol swab on iv pump? :  {'yes': 3, 'no': 3}
is this nurse putting the inhaler on the patient?? :  {'yes': 9, 'no': 4}
is this nurse taking the patient's pulse? :  {'yes': 4, 'no': 4}
is this nurse taking the patient's temperature? :  {'yes': 1, 'no': 1}
is this nurse listening to the patien